In [ ]:
!pip install google-cloud-aiplatform==1.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.2
    Uninstalling shapely-2.0.2:
      Successfully uninstalled shapely-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
!pip install vertexai

In [ ]:
!pip install sql_metadata

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=24df9c5f29a4c1cf584ddf365adf7f46b697a52ee140ea8d0559b11f2dbfa72e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:

import pandas as pd


file_path = '/content/drive/MyDrive/loreal/sql_samples.xlsx'

# Read the Excel file into a pandas DataFrame

df_sample = pd.read_excel(file_path, sheet_name='dataset',header=1)
# Display the DataFrame
q_sql=df_sample.iloc[:, 0:2][0:102]


sql_schema=df_sample.iloc[:, 0:2][103:]
sql_schema.columns=['Schema','type']

In [ ]:
import collections
import json
import os
import re
import sqlite3

from transformers import AutoTokenizer
from sql_metadata import Parser
import pandas as pd
import re
from sql_metadata import Parser

def isFloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def isNegativeInt(value):
    try:
        int_val = int(value)
        return int_val < 0
    except ValueError:
        return False

def sql2skeleton(sql_queries, schema_df):
    column_names = schema_df['Schema'].str.lower().tolist()


    skeletons = []
    for sql in sql_queries:
        parsed_sql = Parser(sql)
        new_sql_tokens = []

        for token in parsed_sql.tokens:
            token_value = token.value.lower()
            # mask column names
            if token_value in column_names:
                new_sql_tokens.append("_")
            # mask string values
            elif token.value.startswith("'") and token.value.endswith("'"):
                new_sql_tokens.append("_")
            # mask positive int number
            elif token.value.isdigit():
                new_sql_tokens.append("_")
            # mask negative int number
            elif isNegativeInt(token.value):
                new_sql_tokens.append("_")
            # mask float number
            elif isFloat(token.value):
                new_sql_tokens.append("_")
            else:
                new_sql_tokens.append(token.value.strip())

        sql_skeleton = " ".join(new_sql_tokens)
        # Additional cleanup can be added here if necessary
        skeletons.append(sql_skeleton)

    return skeletons


# Example usage
sql_queries = ["SELECT SUM(ORDERED_REVENUE) AS ORDERED_REVENUE FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE DIV_DIVISION = 'ACD' AND SYNDICATED_YEAR_NAME = 2023"]


skeletons = sql2skeleton(sql_queries, sql_schema)
print(skeletons)

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

def find_similar_questions(target_question, question_list, top_k=5):
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device="cpu")

    # Encode the target question and all other questions
    target_embedding = model.encode([target_question])
    question_embeddings = model.encode(question_list)

    # Calculate Euclidean distances
    distances = np.squeeze(euclidean_distances(target_embedding, question_embeddings)).tolist()
    distance_indices = list(range(len(question_list)))

    # Pair distances with their indices and sort them
    pairs = sorted(zip(distances, distance_indices), key=lambda x: x[0])

    # Select top K similar questions
    top_k_pairs = pairs[:top_k]
    similar_questions = [question_list[index] for _, index in top_k_pairs]

    return similar_questions

# Example usage
sql_questions = q_sql['Natural Language Question'].tolist()
target_sql_question = q_sql['Natural Language Question'][1]

top_similar_questions = find_similar_questions(target_sql_question, sql_questions, top_k=5)
print(top_similar_questions)

def jaccard_similarity(skeleton1, skeleton2):
    tokens1 = skeleton1.strip().split(" ")
    tokens2 = skeleton2.strip().split(" ")
    total = len(tokens1) + len(tokens2)

    def list_to_dict(tokens):
        token_dict = collections.defaultdict(int)
        for t in tokens:
            token_dict[t] += 1
        return token_dict
    token_dict1 = list_to_dict(tokens1)
    token_dict2 = list_to_dict(tokens2)

    intersection = 0
    for t in token_dict1:
        if t in token_dict2:
            intersection += min(token_dict1[t], token_dict2[t])
    union = (len(tokens1) + len(tokens2)) - intersection
    return float(intersection) / union
def select_top_question_by_jaccard(q_sql, sql_schema, indices, threshold=0.5):
    skeletons = sql2skeleton(q_sql['SQL Query'], sql_schema)
    selected_skeletons = [skeletons[i] for i in indices]

    top_question_indices = []

    for i, skeleton in enumerate(selected_skeletons[1:], start=1):
        similarity = jaccard_similarity(selected_skeletons[0], skeleton)

        if similarity >= threshold:
            top_question_indices.append(indices[i])

    top_questions = [(q_sql['Natural Language Question'][i], q_sql['SQL Query'][i]) for i in top_question_indices]
    return top_questions

# Example usage
# Assuming q_sql is your DataFrame, sql_schema is your schema, and indices is the list of indices of questions
sql_questions = q_sql['Natural Language Question'].tolist()
indices = []

for question in top_similar_questions:
    try:
        index = sql_questions.index(question)
        indices.append(index)
    except ValueError:
        print(f"Question '{question}' not found")

# Now 'indices' contains the indices of the similar questions
q_sql['SQL Query'][indices]
top_question_pairs = select_top_question_by_jaccard(q_sql, sql_schema, indices)
top_question_pairs
class QuestionSqlExampleStyle(object):
    """Provide QA pair as examples"""

    def get_example_prefix(self):
        return "/* Some SQL examples are provided based on similar problems: */\n"

    def format_example(self, example: dict):
        template_qa = "/* Answer the following: {} */\n{}"
        return template_qa.format(example['question'], example['query'])

def format_questions_sql_pairs(question_sql_pairs):
    formatter = QuestionSqlExampleStyle()
    formatted_examples = formatter.get_example_prefix()

    for question, sql in question_sql_pairs:
        example = {'question': question, 'query': sql}
        formatted_examples += formatter.format_example(example) + "\n\n"

    return formatted_examples


formatted_output = format_questions_sql_pairs(top_question_pairs)
print(formatted_output)
def create_table_statement(schema_df, table_name,example):

    # Start the CREATE TABLE statement
    sql = '/* Given the following database schema : */\n'+f"CREATE TABLE {table_name} (\n"

    # Add column definitions
    column_defs = []
    template_question =  "/* Answer the following: {} */"
    prompt_question =template_question.format(example)
    for index, row in schema_df.iterrows():
        column_name = row['Schema']
        data_type = row['type']

        # # Map the data type from DataFrame to SQL data type if necessary
        # if data_type.upper() == 'STRING':
        #     data_type = 'VARCHAR(255)'  # or 'TEXT', depending on your SQL dialect

        # Append the column definition to the list
        column_defs.append(f"    {column_name} {data_type}")

    # Combine all column definitions and add to the statement
    sql += ",\n".join(column_defs)

    # End the CREATE TABLE statement
    sql += "\n);"+prompt_question + "\nA: SELECT "


    return sql

table_schema=create_table_statement(sql_schema, table_name='amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard',example=target_sql_question)

print(table_schema)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


['SELECT SUM ( _ ) AS _ FROM amer - amzaccelerator - data - us - qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE _ = _ AND _ = _']


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

['What is the total sales for each brand in YTD?', 'Top 5 brands, units sold, FC? ', 'What were the top 5 products with the highest YTD ordered revenue in the Skin Care category in the ACD division?', 'Which product in skinc had the highest ordered revenue ytd?', 'What are the top 3 brands with the largest increase in units sold between 21 and 22 in Cosmetics?']
/* Some SQL examples are provided based on similar problems: */
/* Answer the following: Top 5 brands, units sold, FC?  */
SELECT DIV_BRAND, SUM(ORDERED_UNITS) AS ORDERED_UNITS FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE DIV_MASTER_CATEGORY = 'FACIAL SKINCARE' AND SYNDICATED_YEAR_NAME = 2023 GROUP BY DIV_BRAND ORDER BY ORDERED_UNITS DESC LIMIT 5

/* Answer the following: What were the top 5 products with the highest YTD ordered revenue in the Skin Care category in the ACD division? */
SELECT DIV_PRODUCT_TITLE, SUM(ORDERED_REVENUE) AS TOTAL_ORDERED_REVENUE FROM amer-amzaccel

In [ ]:
def combined_function(q_sql, sql_schema,  target_question_index, top_k=5, threshold=0.3):
    # Generate SQL skeletons
    sql_questions = q_sql['Natural Language Question'].tolist()

    target_sql_question=q_sql['Natural Language Question'][target_question_index]
    top_similar_questions = find_similar_questions(target_sql_question, sql_questions, top_k)
    indices = []
    for question in top_similar_questions:
            index = sql_questions.index(question)
            indices.append(index)
    top_question_pairs = select_top_question_by_jaccard(q_sql, sql_schema, indices,threshold)
    # Format the question-SQL pairs
    formatted_output = format_questions_sql_pairs(top_question_pairs)
    table_schema=create_table_statement(sql_schema, table_name='amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard',example=target_sql_question)
    final_prompt=formatted_output+table_schema


    return final_prompt

# Example usage
# Assuming q_sql is your DataFrame of questions and answers, and sql_schema is your schema DataFrame
formatted_output = combined_function(q_sql, sql_schema, target_question_index=2)
print(formatted_output)


/* Some SQL examples are provided based on similar problems: */
/* Answer the following: How many skus does LRP have? */
SELECT COUNT (1) FROM (SELECT DISTINCT(DIV_PRODUCT_TITLE) FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE DIV_BRAND = 'LA ROCHE-POSAY')

/* Answer the following: Highest asp, FR? */
SELECT DIV_PRODUCT_TITLE, AVG(AVERAGE_ORDERED_REVENUE_PRICE) AS AVG_PRICE FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE DIV_MASTER_CATEGORY = 'FRAGRANCE' AND SYNDICATED_YEAR_NAME = 2023 GROUP BY DIV_PRODUCT_TITLE ORDER BY AVG_PRICE DESC LIMIT 1

/* Answer the following: What is the PPM YTD for LRP Thermal Spring Water? */
SELECT AVG(YTD_RETAIL_MARGIN) FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE DIV_PRODUCT_TITLE = 'La Roche-Posay Thermal Spring Water, 10.1 Fl oz.' AND SYNDICATED_YEAR_NAME = 2023

/* Answer the following: Which produ

how to get a dictionary match between columne name and column in question

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()
import vertexai
from vertexai.preview.language_models import TextGenerationModel
PROJECT_ID='cv2023fall'
def predict_large_language_model_sample(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ) :
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
      model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    # print(f"Response from Model: {response.text}")
    return response.text
test_out=predict_large_language_model_sample(PROJECT_ID, "text-bison@001", 0.2, 256, 0.8, 40, '''Give me ten interview questions for the role of program manager''', "us-central1")

In [ ]:
test_out

'1. What is your experience with project management?\n2. What are your strengths and weaknesses as a project manager?\n3. How do you handle conflict and difficult situations?\n4. How do you stay organized and on top of your work?\n5. What is your communication style?\n6. How do you work with stakeholders?\n7. How do you manage your time effectively?\n8. What are your goals for your career?\n9. Why are you interested in this position?\n10. What are your salary expectations?'

In [ ]:
prediction=[]
for i in range(q_sql.shape[0]):
    formatted_output=combined_function(q_sql, sql_schema, target_question_index=i)

# Call the function with the new prompt
    output=predict_large_language_model_sample(PROJECT_ID, "text-bison@001", 0.2, 500, 0.8, 40, formatted_output, "us-central1")

    output='SELECT '+ re.sub(' +', ' ', output.replace("\n", " ")).strip()

    print(f'{1+i},{output}')
    prediction.append(output)

1,SELECT SUM(ORDERED_REVENUE) AS ORDERED_REVENUE FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE DIV_DIVISION = 'ACD' AND SYNDICATED_YEAR_NAME = 2023
2,SELECT div_brand, sum(ordered_revenue) as total_sales FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE div_master_category = 'FACIAL SKINCARE' AND SYNDICATED_YEAR_NAME = 2023 GROUP BY div_brand ORDER BY total_sales DESC
3,SELECT AVG(AVERAGE_ORDERED_REVENUE_PRICE) FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE DIV_BRAND = 'LA ROCHE-POSAY'
4,SELECT MIN(SYNDICATED_MONTH) FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE DIV_PRODUCT_TITLE = 'La Roche-Posay Thermal Spring Water, 10.1 Fl oz.'
5,SELECT PPM_AMZR FROM amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard WHERE DIV_PRODUCT_TITLE = 'La Roche-Posay T

In [ ]:
def calculate_exact_match(generated_sql, ground_truth_sql):
    # Normalize the SQL queries (optional, but recommended)
    normalized_generated_sql = generated_sql.strip().lower()
    normalized_ground_truth_sql = ground_truth_sql.strip().lower()

    # Compare for exact match
    return normalized_generated_sql == normalized_ground_truth_sql

# Example usage

exact_match_score=0
for i in range(q_sql.shape[0]):
    exact_match = calculate_exact_match(prediction[i],q_sql['SQL Query'].values[i] )
    exact_match_score +=exact_match


exact_match_score

7

In [ ]:
7/102

In [ ]:
with open("/content/drive/MyDrive/loreal/palm_prediction.txt", "w") as file:
    for item in prediction:
        file.write(item + "\n")

In [6]:
qm_path = "/content/drive/MyDrive/loreal/wenxi_zhang/data/palm_prediction.txt"
with open(qm_path, 'r') as file:
    # Read lines into a list, stripping the newline character from each line
    lines = [line.strip() for line in file]
ground_tru=[]
palm=[]
for i in q_sql['SQL Query']:
  gt=i.replace("amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard", "df")
  ground_tru.append(gt)

for i in lines:
  gt=i.replace("amer-amzaccelerator-data-us-qa.amztech_bqdset_us_qa_view.view_gold_commercial_dashboard", "df")
  palm.append(gt)
import sqlite3
import pandas as pd
import pyarrow.parquet as pq
parquet_file = '/content/drive/My Drive/loreal/anonymized_amz_data_dashboard.parquet'
file_path = parquet_file
table = pq.read_table(file_path)

def convert_dbdate(data):
    return data.astype(str)

converted_columns = {field.name: convert_dbdate(table[field.name]) if 'dbdate' in str(field.type) else table[field.name] for field in table.schema}

df = pd.DataFrame(converted_columns)


df.shape
conn = sqlite3.connect('my_database.db')
# Write the data to a SQLite table
df.to_sql('df', conn, if_exists='replace', index=False)
result=[]
for te in ground_tru:
  try:
    cursor = conn.cursor()
    cursor.execute(te)
    result_from_ground_truth = cursor.fetchall()
    cursor.close()
  except Exception as e:
        result_from_ground_truth = str(e)
  result.append(result_from_ground_truth)
palm_result=[]
for q in palm:
    try:
      cursor = conn.cursor()
      cursor.execute(q)
      result_from_p = cursor.fetchall()
      cursor.close()
    except Exception as e:
      result_from_p = str(e)
    palm_result.append(result_from_p)
successful_executions=0
for i,j in zip(result,palm_result):
  if i and j:
    if i == j:
        successful_executions += 1

In [7]:
successful_executions

26

In [8]:
successful_executions/102

0.25742574257425743